## Created by yunsuxiaozi

### Import necessary libraries

In [ ]:
# https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-training
# https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-inference

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import timm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import random
import warnings
warnings.filterwarnings('ignore')

### Config

In [ ]:
class Config:
    seed = 2024
    image_transform = transforms.Resize((512, 512))
    batch_size = 64
    num_epochs = 10
    num_folds = 5

### Seed 

In [ ]:
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything(Config.seed)

### labels

In [ ]:
train_df=pd.read_csv("data/train.csv")

labels=['seizure','lpd','gpd','lrda','grda','other']
for label in labels:
    group=train_df[f'{label}_vote'].groupby(train_df['spectrogram_id']).sum()
    label_vote_sum = pd.DataFrame({'spectrogram_id': group.index, f'{label}_vote_sum': group.values})
    if label=='seizure':
        train_feats=label_vote_sum
    else:
        train_feats=train_feats.merge(label_vote_sum,on='spectrogram_id',how='left')
train_feats['total_vote']=0
for label in labels:
      train_feats['total_vote']+=train_feats[f'{label}_vote_sum']
for label in labels:
      train_feats[f'{label}_vote']=train_feats[f'{label}_vote_sum']/train_feats['total_vote']
choose_cols=['spectrogram_id']
for label in labels:
    choose_cols+=[f'{label}_vote']
train_feats=train_feats[choose_cols]
train_feats['path'] = train_feats['spectrogram_id'].apply(lambda x: "data/train_spectrograms/" + str(x) + ".parquet" )
train_feats.head()

### Loss function :Kullback Leibler Divergence

In [ ]:
def KL_loss(p,q):
    epsilon=10**(-15)
    p=torch.clip(p,epsilon,1-epsilon)
    q = nn.functional.log_softmax(q,dim=1)
    return torch.mean(torch.sum(p*(torch.log(p)-q),dim=1))

### get_batch_data

In [ ]:
def get_batch(paths,batch_size=Config.batch_size):
    eps=1e-6
    batch_data=[]
    for path in paths:
        data=pd.read_parquet(path[0])
        data = data.fillna(-1).values[:, 1:].T
        data=data[:, 0:300]
        data=np.clip(data, np.exp(-6), np.exp(10))
        data= np.log(data)
        data_mean=data.mean(axis=(0, 1))
        data_std=data.std(axis=(0, 1))
        data=(data - data_mean)/(data_std + eps)
        data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
        data=Config.image_transform(data_tensor)
        batch_data.append(data)
    batch_data=torch.stack(batch_data)
    return batch_data

### Model_training

In [ ]:
device ='cuda' if  torch.cuda.is_available() else 'cpu'
print(f"device:{device}")


total_idx=np.arange(len(train_feats))
np.random.shuffle(total_idx)

for fold in range(Config.num_folds):

    test_idx=total_idx[fold*len(total_idx)//Config.num_folds:(fold+1)*len(total_idx)//Config.num_folds]
    train_idx=np.array([idx for idx in total_idx if idx not in test_idx])

    model = timm.create_model('resnet34d',pretrained=True, num_classes=6, in_chans=1)
    
    optimizer=optim.AdamW(model.parameters(),lr=0.001,betas=(0.5,0.999),weight_decay=0.01)

    best_test_loss=1.0
    train_losses=[]
    test_losses=[]

    print(f"start")

    for epoch in range(Config.num_epochs):
        print(f"epoch {epoch}:")
        model.to(device)
        model.train()
        train_loss=[]
        random_num=np.arange(len(train_idx))
        np.random.shuffle(random_num)
        train_idx=train_idx[random_num]

        for idx in range(0,len(train_idx),Config.batch_size): 
            optimizer.zero_grad()
            train_idx1=train_idx[idx:idx+Config.batch_size]
            train_X1_path=train_feats[['path']].iloc[train_idx1].values
            train_X1=get_batch(train_X1_path,batch_size=Config.batch_size)
            train_y1=train_feats[['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']].iloc[train_idx1].values
            train_y1=torch.Tensor(train_y1)

            train_pred=model(train_X1.to(device)).to(device)

            loss=KL_loss(train_y1.to(device),train_pred.to(device)).to(device)

            loss.backward()

            optimizer.step()
            print(f"idx:{idx},loss:{loss}")
            train_loss.append(loss.detach().cpu().numpy())
        train_loss=np.mean(np.array(train_loss))
        print(f"train_loss:{train_loss}")
        test_loss=[]
        model.eval()
        with torch.no_grad():
            for idx in range(0,len(test_idx),Config.batch_size): 
                test_idx1=test_idx[idx:idx+Config.batch_size]
                test_X1_path=train_feats[['path']].iloc[test_idx1].values
                test_X1=get_batch(test_X1_path,batch_size=Config.batch_size)

                test_y1=train_feats[['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']].iloc[test_idx1].values
                test_y1=torch.Tensor(test_y1)

                test_pred=model(test_X1.to(device)).to(device)

                loss=KL_loss(test_y1.to(device),test_pred.to(device)).to(device)
                test_loss.append(loss.detach().cpu().numpy())
        test_loss=np.mean(np.array(test_loss))
        print(f"test_loss:{test_loss}")
        if test_loss<best_test_loss:
            best_test_loss=test_loss
            torch.save(model.to('cpu'),f"HMS_resnet_fold{fold}.pth")
            
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print("-"*50)
    print(f"best_test_loss:{best_test_loss}") 
    plt.title("train_losses VS test_losses")
    epochs=[i for i in range(len(train_losses))]
    plt.plot(epochs,train_losses,marker="o",markersize=1,label="train_losses")
    plt.plot(epochs,test_losses,marker="x",markersize=1,label="test_losses")
    plt.legend()
    plt.show()